In [ ]:
## TO DO NEXT:
# Rename/ Pivot all the ABS lending data together for analysis (continue)
# Need to clean up the existing code + rename dictionaries + tables

# Later: Github

# Error handling: how to know if file has changed or is missing columns
# When importing, save archived copies
# Do a check where for each table summary stats are created. i.e. average, min/max etc.
# Compare these across archives

# Import Libraries
import pandas as pd # Data Analysis Library
import matplotlib.pyplot as plt # Data Visualisation Library
import matplotlib.ticker as ticker
%matplotlib inline

import seaborn as sns # Data Visualisation Library
import requests # For downloading files off web
import datetime
import re # Regex

import pickle # for saving/loading files

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Functions:

# Function to save and load dataframes and dictionaries
def pickle_save(name, to_save):
    with open('./Output/' + name + '.pickle', 'wb') as handle:
        pickle.dump(to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def pickle_load(name):
    with open('./Output/' + name + '.pickle', 'rb') as handle:
        load_data = pickle.load(handle)
    return load_data

In [ ]:
# Extract Data
#  TBD
# Sense-Check Extrated Data
#  TBD

In [ ]:
# Use the dictionary with the following structure for the loop
"filename for the download", "rows to keep", "series ID columns to keep"

# still need to readd RBA to this
file_dict = {}
file_dict = { # Dictionary to store filename and filepath mapping
    
    'Credit Card Data - Australia' : {
    'dl_url': 'https://www.rba.gov.au/statistics/tables/xls/',
    'dl_name': 'c01-2-hist',
    'dl_source': 'RBA',
    'readable_name': 'RBA-Credit-Card-Total',
    'sheet_name': 'Data',
    'filetype': '.xlsx',
    'row_drop': ['Description', 'Frequency', 'Type', 'Units', 'Source', 'Publication date', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Units', # Name of row that contains units
    'series': ['CCCCSNAP', 'CCCCSBAIP', 'CCCCSBTP', 'CCCCSCLP', 'CCCCSTTVP'], # need to review
    'publication_row': 'Publication date',
    'skiprow': 1
    },
    
    'Household Finance - Total' : {
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560101',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-Total',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108295286C','A108299115J','A108276981A','A108299885C','A108271213V','A108294614K'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Total by Lender' : {
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560102',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-Total-Lender',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108286039A','A108312356A','A108286046X','A108276379K','A108312352T','A108285311J','A108272326J','A108312354W',
                'A108267776W'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied by Purpose' : {
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560103',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-Purpose',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108299116K','A108284976J','A108280580R','A108299018F','A108298969V','A108294237R','A108294069R',
                'A108280461X','A108299115J','A108284975F','A108280579F','A108299017C','A108298968T','A108294236L',
                'A108294068L','A108280460W'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied by State' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560104',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-State',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108299116K','A108271760F','A108289778K','A108267070V','A108299074X','A108299088L','A108280692J',
                'A108299109L','A108285277R','A108299115J','A108271759W','A108289777J','A108267069K','A108299073W',
                'A108299087K','A108280691F','A108299108K','A108285276L'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },

    'Household Finance - Owner Occupied - NSW' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560105',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-NSW',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108271760F','A108271403J','A108294300V','A108271662A','A108271578K','A108276100R','A108289421V',
                'A108284899R','A108271759W','A108271402F','A108294299W','A108271661X','A108271577J','A108276099T',
                'A108289420T','A108284898L'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied - VIC' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560106',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-VIC',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108289778K','A108284955W','A108271501L','A108276240T','A108271557X','A108276072T','A108266601A',
                'A108266664K','A108289777J','A108284954V','A108271500K','A108276239J','A108271556W','A108276071R',
                'A108266600X','A108266663J'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied - QLD' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560107',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-QLD',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108267070V','A108284948X','A108266825L','A108271641R','A108266888W','A108294223A','A108275911A',
                'A108298752F','A108267069K','A108284947W','A108266824K','A108271640L','A108266887V','A108294222X',
                'A108275910X','A108298751C'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied - SA' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560108',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-SA',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108299074X','A108294139K','A108298871W','A108276212J','A108276149C','A108289533L','A108298682K',
                'A108298710J','A108299073W','A108294138J','A108298870V','A108276211F','A108276148A','A108289532K',
                'A108298681J','A108298709X'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied - WA' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560109',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-WA',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108299088L','A108271368L','A108298885K','A108266923T','A108280601R','A108280510J','A108289386X',
                'A108284871L','A108299087K','A108271367K','A108298884J','A108266922R','A108280600L','A108280509X',
                'A108289385W','A108284870K'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied - TAS' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560110',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-TAS',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108280692J','A108298780R','A108266783A','A108271606J','A108298941T','A108280503K','A108294041L',
                'A108289449W','A108280691F','A108298779F','A108266782X','A108271605F','A108298940R','A108280502J',
                'A108294040K','A108289448V'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied - NT' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560111',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-NT',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108299109L','A108276016X','A108280573T','A108280664X','A108271571V','A108276086F','A108275918T',
                'A108289456V','A108299108K','A108276015W','A108280572R','A108280663W','A108271570T','A108276085C',
                'A108275917R','A108289455T'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupied - ACT' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560112',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-OO-ACT',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108285277R','A108294167V','A108266804A','A108271620C','A108266867K','A108276058W','A108266587T',
                'A108271312A','A108285276L','A108294166T','A108266803X','A108271619V','A108266866J','A108276057V',
                'A108266586R','A108271311X'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor by Purpose' : {
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560113',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-Purpose',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108276982C','A108276723K','A108285662K','A108272187V','A108299536F','A108294839X','A108299284W',
                'A108289981T','A108276981A','A108276722J','A108285661J','A108272186T','A108299535C','A108294838W',
                'A108299283V','A108289980R'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor by State' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560114',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-State',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108276982C','A108290331L','A108299732R','A108272299L','A108272257R','A108267742X','A108290303C',
                'A108267763K','A108295077J','A108276981A','A108290330K','A108299731L','A108272298K','A108272256L',
                'A108267741W','A108290302A','A108267762J','A108295076F'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
       
    'Household Finance - Investor - NSW' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560115',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-NSW',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108290331L','A108280986R','A108285648R','A108299592X','A108272152V','A108281042X','A108294685V',
                'A108271949K','A108290330K','A108280985L','A108285647L','A108299591W','A108272151T','A108281041W',
                'A108294684T','A108271948J'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
        
    'Household Finance - Investor - VIC' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560116',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-VIC',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108299732R','A108267392K','A108281070J','A108285725J','A108285683W','A108290072F','A108271893K',
                'A108294748T','A108299731L','A108267391J','A108281069X','A108285724F','A108285682V','A108290071C',
                'A108271892J','A108294747R'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor - QLD' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560117',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-QLD',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108272299L','A108272005V','A108267511R','A108295000K','A108290170K','A108281028C','A108276541W',
                'A108271942V','A108272298K','A108272004T','A108267510L','A108294999K','A108290169A','A108281027A',
                'A108276540V','A108271941T'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor - SA' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560118',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-SA',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108272257R','A108267350L','A108285620L','A108294958R','A108276821R','A108294783X','A108267252J',
                'A108280916J','A108272256L','A108267349C','A108285619C','A108294957L','A108276820L','A108294782W',
                'A108267251F','A108280915F'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor - WA' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560119',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-WA',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108267742X','A108267364A','A108276758K','A108276877A','A108299508W','A108290044W','A108289918A',
                'A108280930C','A108267741W','A108267363X','A108276757J','A108276876X','A108299507V','A108290043V',
                'A108289917X','A108280929V'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor - TAS' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560120',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-TAS',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108290303C','A108267357C','A108272068C','A108290198L','A108281119K','A108290037X','A108271879R',
                'A108276597J','A108290302A','A108267356A','A108272067A','A108290197K','A108281118J','A108290036W',
                'A108271878L','A108276596F'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor - NT' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560121',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-NT',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108267763K','A108267406L','A108276793T','A108290226K','A108294930L','A108299424L','A108271900W',
                'A108276639V','A108267762J','A108267405K','A108276792R','A108290225J','A108294929C','A108299423K',
                'A108271899X','A108276638T'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor - ACT' : { # Excludes refinancing
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560122',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-ACT',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108295077J','A108267371X','A108276772F','A108281168F','A108294895T','A108299410X','A108280874W',
                'A108294713T','A108295076F','A108267370W','A108276771C','A108281167C','A108294894R','A108299409R',
                'A108280873V','A108294712R'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - FHB by State' : {
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560123',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-FHB',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108299886F','A108295203K','A108285956T','A108299851F','A108290415W','A108281343C','A108272390A',
                'A108281385A','A108285942C','A108299885C','A108295202J','A108285955R','A108299850C','A108290414V',
                'A108281342A','A108272389T','A108281384X','A108285941A'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Owner Occupier - FHB' : {
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560124',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-FHB',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108271214W','A108271207X','A108298654A','A108298647C','A108298612C','A108280335K','A108271179A',
                'A108266545V','A108275841F','A108271213V','A108271206W','A108298653X','A108298646A','A108298611A',
                'A108280334J','A108271178X','A108266544T','A108275840C'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Investor - FHB' : {
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560125',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-INV-FHB',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108294615L','A108289876R','A108280832X','A108294608R','A108271816F','A108276471A','A108276457F',
                'A108289869T','A108294594L','A108294614K','A108289875L','A108280831W','A108294607L','A108271815C',
                'A108276470X','A108276456C','A108289868R','A108294593K'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    },
    
    'Household Finance - Non-Resident' : {
    'dl_url': 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/',
    'dl_name': '560126',
    'dl_source': 'ABS',
    'readable_name': 'ABS-Housing-NON-RESI',
    'sheet_name': 'Data1',
    'filetype': '.xls',
    'row_drop': ['Unit', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'No. Obs',
                'Series Start', 'Series End', 'Series ID'], # Rows to drop
    'ID_row': 'Series ID', # Name of row that contains the unique ID
    'unit_row': 'Unit', # Name of row that contains units
    'series':  ['A108289120R','A108289119F'], # need to review
    'publication_row': 'Series End',
    'skiprow': 0
    }

}

In [ ]:
for key in file_dict:
    dl_name = file_dict[key]['dl_name'] + file_dict[key]['filetype']
    dl_url = file_dict[key]['dl_url'] + dl_name
    r = requests.get(dl_url)
    with open("./Input/" + dl_name, 'wb') as f:
              f.write(r.content)    

In [ ]:
output_d = {}

for key in file_dict:   
    data_name = file_dict[key]['dl_name']
    row_filter = file_dict[key]['row_drop']
    series_filter = file_dict[key]['series']
    
    read_path = './Input/' + data_name + file_dict[key]['filetype']
    df = pd.read_excel(read_path, sheet_name = file_dict[key]['sheet_name'], skiprows = file_dict[key]['skiprow'])
    df = df.rename(columns={ df.columns[0]: 'Title' })
    
    description = df.columns.values # Series ID Names
    
    series_id_row = df[df['Title'] == 'Series ID'].values.tolist()[0]
    series_id_row[0] = 'Title'

    df.columns = series_id_row
    
    # Extract Publication Date
    pub_date = df.loc[df['Title'] == file_dict[key]['publication_row']].values[0,1]
    if type(pub_date) == datetime.datetime:
        pub_date = pub_date.strftime("%d-%b-%Y")
    print(file_dict[key]['readable_name'], 'latest publication date:', pub_date) #loc = index check
    
    # Transform Data 
    series_id = df[df['Title'] == file_dict[key]['ID_row']].values[0] # Series ID values
    units = df[df['Title'] == file_dict[key]['unit_row']].values[0] # Unit values
    series_lookup = pd.DataFrame(list(zip(series_id,description,units)), columns=['Series ID','Description','Unit'])
    series_lookup = series_lookup[series_lookup['Series ID'] != 'Series ID'] # Need to save these in the dictionary output
    series_lookup = series_lookup.loc[series_lookup['Series ID'].isin(series_filter), :]
    
    # select the series_to_filter
    column_filter = series_lookup.loc[:,'Series ID'].values.tolist()
    column_filter.insert(0, 'Title')
    df_fmt = df.loc[:,column_filter] # Keep relevant series

    # Drop Rows
    df_fmt = df_fmt[~df_fmt['Title'].isin(row_filter)]
    df_fmt = df_fmt.dropna(subset=['Title']) # Drop these rows 
    
    df_fmt = df_fmt.reset_index(drop=True)
    df_fmt = df_fmt.rename(columns={'Title': 'Date'})
    df_fmt = df_fmt.convert_dtypes() # Convert variable types
    
    output_d[key] = {}
    output_d[key]['Data'] = df_fmt 
    output_d[key]['Lookup'] = series_lookup

In [ ]:
# Save the imported data
pickle_save('imported_data',output_d)